In [21]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
from scipy.interpolate import interp1d
import math
from features import implied_probability, american_to_decimal, fixed_length, split_for_datetime, time_for_75, fixed_length_timelin, linspace_stamps 

In [22]:
df = pd.read_csv("timestampsDatetimeA.csv", header=None, names=range(231))
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True, inplace=True)
df.head()

C:\Users\maxbr\AppData\Local\Temp\ipykernel_13856\2356563460.py:1: DtypeWarning: Columns (206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("timestampsDatetimeA.csv", header=None, names=range(231))


,fighter,event,date,opponent,timestep-1,timestep-2,timestep-3,timestep-4,timestep-5,timestep-6,...,timestep-218,timestep-219,timestep-220,timestep-221,timestep-222,timestep-223,timestep-224,timestep-225,timestep-226,timestep-227
0,Mansur Abdul-Malik,UFC,2025-02-23,Nick Klein,2025-02-13 14:17:00 - price: -549,2025-02-13 20:33:00 - price: -549,2025-02-14 18:09:00 - price: -562,2025-02-14 18:13:00 - price: -532,2025-02-14 19:09:00 - price: -569,2025-02-14 19:11:00 - price: -569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Daichi Abe,UFC Fight Night 132: Cowboy vs. Edwards,2018-06-23,Li Jingliang,2018-06-13 12:09:00 - price: +240,2018-06-13 12:21:00 - price: +325,2018-06-13 13:42:00 - price: +315,2018-06-13 15:24:00 - price: +323,2018-06-13 16:18:00 - price: +320,2018-06-13 17:06:00 - price: +313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,John Adajar,Road to UFC 4,2022-06-10,Han Seul Kim,2022-06-07 10:00:00 - price: -120,2022-06-07 10:18:00 - price: +105,2022-06-07 22:02:00 - price: -105,2022-06-08 13:08:00 - price: +100,2022-06-08 13:26:00 - price: +115,2022-06-08 13:50:00 - price: +118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Shamil Abdurakhimov,UFC 283,2023-01-21,Jailton Almeida,2023-01-06 10:00:00 - price: +600,2023-01-06 13:20:00 - price: +583,2023-01-07 19:48:00 - price: +588,2023-01-08 15:40:00 - price: +585,2023-01-08 23:16:00 - price: +593,2023-01-08 23:54:00 - price: +596,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hamdy Abdelwahab,UFC,2025-02-01,Jamal Pogues,2025-01-14 07:53:00 - price: +105,2025-01-14 12:39:00 - price: +113,2025-01-15 08:41:00 - price: +110,2025-01-16 12:13:00 - price: +111,2025-01-16 14:27:00 - price: +111,2025-01-16 15:57:00 - price: +112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df = df.dropna(thresh=4)
df = df.iloc[:, 4:]
df.head()

,timestep-1,timestep-2,timestep-3,timestep-4,timestep-5,timestep-6,timestep-7,timestep-8,timestep-9,timestep-10,...,timestep-218,timestep-219,timestep-220,timestep-221,timestep-222,timestep-223,timestep-224,timestep-225,timestep-226,timestep-227
0,2025-02-13 14:17:00 - price: -549,2025-02-13 20:33:00 - price: -549,2025-02-14 18:09:00 - price: -562,2025-02-14 18:13:00 - price: -532,2025-02-14 19:09:00 - price: -569,2025-02-14 19:11:00 - price: -569,2025-02-15 21:29:00 - price: -596,2025-02-16 02:57:00 - price: -608,2025-02-16 08:25:00 - price: -636,2025-02-16 12:01:00 - price: -667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-06-13 12:09:00 - price: +240,2018-06-13 12:21:00 - price: +325,2018-06-13 13:42:00 - price: +315,2018-06-13 15:24:00 - price: +323,2018-06-13 16:18:00 - price: +320,2018-06-13 17:06:00 - price: +313,2018-06-13 20:45:00 - price: +318,2018-06-13 23:54:00 - price: +315,2018-06-14 01:36:00 - price: +308,2018-06-14 09:18:00 - price: +305,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-06-07 10:00:00 - price: -120,2022-06-07 10:18:00 - price: +105,2022-06-07 22:02:00 - price: -105,2022-06-08 13:08:00 - price: +100,2022-06-08 13:26:00 - price: +115,2022-06-08 13:50:00 - price: +118,2022-06-08 13:54:00 - price: +123,2022-06-08 15:36:00 - price: +122,2022-06-09 15:36:00 - price: +125,2022-06-09 15:52:00 - price: +130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-01-06 10:00:00 - price: +600,2023-01-06 13:20:00 - price: +583,2023-01-07 19:48:00 - price: +588,2023-01-08 15:40:00 - price: +585,2023-01-08 23:16:00 - price: +593,2023-01-08 23:54:00 - price: +596,2023-01-09 03:14:00 - price: +596,2023-01-09 11:30:00 - price: +603,2023-01-09 11:56:00 - price: +603,2023-01-10 15:46:00 - price: +588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-14 07:53:00 - price: +105,2025-01-14 12:39:00 - price: +113,2025-01-15 08:41:00 - price: +110,2025-01-16 12:13:00 - price: +111,2025-01-16 14:27:00 - price: +111,2025-01-16 15:57:00 - price: +112,2025-01-19 09:11:00 - price: +112,2025-01-20 16:19:00 - price: +112,2025-01-20 19:35:00 - price: +111,2025-01-20 20:21:00 - price: +108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
X = []
y = []
checklist = []

for index, row in df.iterrows():
    dfTest = df.loc[[index]]
    dfHold = dfTest

    dfTest = dfTest.dropna(axis=1)

    split_name = time_for_75(dfTest)
    split_index = dfTest.columns.get_loc(split_name)

    timestamps = linspace_stamps(dfTest)


    dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
            if isinstance(x, str) and 'price: ' in x else x
        )
    

    if split_index <8:
        continue
    
    target = float(dfTest.iloc[0, -1])
    sequence = dfTest.iloc[0, :split_index].values
    sequence = np.array(sequence, dtype=float)

    timestamps_seq = timestamps.iloc[0, :split_index].values
    timestamps_seq = np.array(timestamps_seq, dtype=float)
    
    last_value = sequence[-1]

    #0 means take the 48hr
    if target < last_value:
        target = 0
    #1 means take the closing 
    else:
        target = 1

    #decimal_seq = american_to_decimal(sequence)
    #impliedProb_seq = implied_probability(sequence)
    
    interpolated_seq = fixed_length_timelin(sequence, 100, 'linear', timestamps_seq)
    interpolated_IP = implied_probability(interpolated_seq)

    #interpolated_dec = fixed_length(decimal_seq, 20, 'linear')
    #interpolated_imp = fixed_length(impliedProb_seq, 20, 'linear')

    #interpolated_all = np.concatenate((interpolated_seq, interpolated_dec, interpolated_imp))
    X.append(interpolated_IP)
    y.append(target)

C:\Users\maxbr\AppData\Local\Temp\ipykernel_13856\3856474640.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_13856\3856474640.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_13856\3856474640.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_13856\3856474640.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  dfTest = dfTest.applymap(lambda x: x.split(' - price: ')[1].strip()
C:\Users\maxbr\AppData\Local\Temp\ipykernel_13856\3856474640.py:17: FutureWarning: DataFrame.applymap has been deprecate

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train[0]))

100


In [26]:
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [27]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6049


In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.49      0.54       992
           1       0.61      0.71      0.65      1101

    accuracy                           0.60      2093
   macro avg       0.60      0.60      0.60      2093
weighted avg       0.60      0.60      0.60      2093



In [30]:
import joblib
joblib.dump(model, 'timeLinearNormalized100.pkl')

['timeLinearNormalized100.pkl']